#### Notes:
1. Python ASN.1 tools library does not yet support parameterization, meaning DATE-TIME-FRACTIONAL isn't accepted. Test examples without it for now.
2. Schema is compiled from file, but message to be encoded currently must be input as a python dict.

In [1]:
import numpy as np
from datetime import datetime
import asn1tools
import asn1
import pyasn1
#import asn1vnparser

## Compile schema from file

In [2]:
schema_path = '/Users/michaelbondin/ASN1-Rad-Data-Streaming-Standard/standard_schema_complete_without_date-time-fractional.asn'
encoding_rule = 'der'
spec = asn1tools.compile_files(schema_path, encoding_rule)

## Provide message as python dict

#### Example Gross Gamma Counts Radiation Data Packet

In [60]:
message = {
    'radiationDataPacketNumber'   : 1,
    'timeInformationRadiationData' :
    {
        'relativeTime'             : 116011,
        'accumulationTime'         : 112911
    },
    'detectorsListRadiationData'   : [
        {
            'detectorID'           : 0,
            'counts'               : 273058,
            'channelSpectrum'      : [587, 9339, 13919, 6590, 6821, 7008, 7366, 7930, 
                                    8137, 8014, 7964, 9313, 10812, 9506, 8446, 7484, 
                                    6757, 6314, 5971, 5680, 5363, 5243, 5246, 5135, 
                                    5099, 5166, 5433, 5577, 5703, 4835, 2862, 1793, 
                                    1306, 900, 728, 615, 477, 510, 787, 3366, 14696, 
                                    24910, 9902, 935, 214, 191, 161, 157, 171, 189, 
                                    160, 169, 135, 144, 118, 115, 111, 110, 91, 89, 
                                    86, 64, 38, 587, 9339, 13919, 6590, 6821, 7008, 7366, 7930, 
                                    8137, 8014, 7964, 9313, 10812, 9506, 8446, 7484, 
                                    6757, 6314, 5971, 5680, 5363, 5243, 5246, 5135, 
                                    5099, 5166, 5433, 5577, 5703, 4835, 2862, 1793, 
                                    1306, 900, 728, 615, 477, 510, 787, 3366, 14696, 
                                    24910, 9902, 935, 214, 191, 161, 157, 171, 189, 
                                    160, 169, 135, 144, 118, 115, 111, 110, 91, 89, 
                                    86, 64, 38],
            'derivedData'          :
            {
                'countRate'        : 2744.299263,
                'deadTime'         : 6.375924139,
            }
        }
    ]
}

# message = {
#     'radiationDataPacketNumber'    : 3,
#     'timeInformationRadiationData' : {'relativeTime'     : 3010,
#                                       'accumulationTime' : 2000},
#     'detectorsListRadiationData'   : [{'detectorID'  : 0,
#                                        'counts'      : 6489,
#                                        'derivedData' : {'countRate'    : 3244.5,
#                                                         'exposure'     : 40.056,
#                                                         'exposureRate' : 72100.0,
#                                                         'alarm1'       : True,
#                                                         'deadTime'     : 10.0,
#                                                         'liveTime'     : 1800.0}
#                                        }],
#     'endRadiationDataPacket'       : 1110010
# }

message

{'radiationDataPacketNumber': 1,
 'timeInformationRadiationData': {'relativeTime': 116011,
  'accumulationTime': 112911},
 'detectorsListRadiationData': [{'detectorID': 0,
   'counts': 273058,
   'channelSpectrum': [587,
    9339,
    13919,
    6590,
    6821,
    7008,
    7366,
    7930,
    8137,
    8014,
    7964,
    9313,
    10812,
    9506,
    8446,
    7484,
    6757,
    6314,
    5971,
    5680,
    5363,
    5243,
    5246,
    5135,
    5099,
    5166,
    5433,
    5577,
    5703,
    4835,
    2862,
    1793,
    1306,
    900,
    728,
    615,
    477,
    510,
    787,
    3366,
    14696,
    24910,
    9902,
    935,
    214,
    191,
    161,
    157,
    171,
    189,
    160,
    169,
    135,
    144,
    118,
    115,
    111,
    110,
    91,
    89,
    86,
    64,
    38,
    587,
    9339,
    13919,
    6590,
    6821,
    7008,
    7366,
    7930,
    8137,
    8014,
    7964,
    9313,
    10812,
    9506,
    8446,
    7484,
    6757,
    6314,
    5

#### Example Gross Gamma Counts Instrument Status Packet

In [54]:
from datetime import datetime

message = {
    'instrumentStatusPacketNumber'    : 2,
    'timeInformationInstrumentStatus' :
    {
        'relativeTime'                : 1050,
        'timeStamp'                   : datetime.fromisoformat('2023-03-20T15:23:27')
    },
    'instrumentStatusInformation'     :
    {
        'instrumentPower'             : ('instrumentPowerStatus', True),
        'instrumentHealth'            : True,
        'instrumentSelfCalibration'   : True
    },
    'detectorsListInformationStatus'  :
    [
        {
            'detectorID'              : 0,
            'detectorHealth'          : True
        }
    ]
}


# message = {
#     'instrumentStatusPacketNumber'     : 2,
#     'timeInformationInstrumentStatus'  : {'relativeTime'     : 1005,
#                                          'timeStamp'        : datetime.fromisoformat('2023-03-20T15:23:27')},
#     'instrumentStatusInformation'      : {'instrumentTemperature' : 23,
#                                          'instrumentPower'       : ('instrumentBatteryRemainingCharge', 70),
#                                          'instrumentHealth'      : True},
#     'detectorsListInformationStatus'   : [{'detectorID'          : 0,
#                                            'detectorVoltage'     : -1290,
#                                            'detectorTemperature' : 34
#                                           }],
#     'endInstrumentStatusPacket'       : 1111000
# }

message

{'instrumentStatusPacketNumber': 2,
 'timeInformationInstrumentStatus': {'relativeTime': 1050,
  'timeStamp': datetime.datetime(2023, 3, 20, 15, 23, 27)},
 'instrumentStatusInformation': {'instrumentPower': ('instrumentPowerStatus',
   True),
  'instrumentHealth': True,
  'instrumentSelfCalibration': True},
 'detectorsListInformationStatus': [{'detectorID': 0, 'detectorHealth': True}]}

#### Example Gross Gamma-Neutron Counts Header Packet

In [55]:
message = {
    'timeInformationHeader' :
    {
        'timeStampZero'                          : datetime.fromisoformat('2023-03-20T15:23:27'),
        'instrumentStatusPacketTransmissionRate' : 0.0167
    },
    'instrumentInformationHeader' : 
    {
        'instrumentName'                         : "Example SGC Detector",
        'instrumentDescription'                  : "Sample Spectroscopic Gamma Detector",
        'instrumentManufacturer'                 : "ANSI",
        'instrumentModel'                        : "SGC 1.00",
        'instrumentSerialNumber'                 : "SGC-00000001",
        'numberOfDetectors'                      : 1
    },
    'detectorsListHeader' :
    [
        {
            'detectorID'                         : 0,
            'detectorType'                       : "spectroscopic",
            'detectorKind'                       : "cLYC",
            'spectrumCompression'                : False,
            'backgroundSubtraction'              : False,
            'numberOfMCAChannels'                : 1000
        }
    ]
}

# message = {
#     'timeInformationHeader'       : 
#         {
#          'timeStampZero'                       : datetime.fromisoformat('2023-03-20T15:23:27'),
#          'radiationDataPacketTransmissionRate' : 2.0
#         },
#     'instrumentInformationHeader' : 
#         {
#         'instrumentName'         : "GN1",
#         'instrumentDescription'  : 'Sample Gross Gamma Counts Detector',
#         'instrumentManufacturer' : 'IEEE',
#         'instrumentModel'        : 'Basic Gamma-Neutron Instrument 1.00',
#         'numberOfDetectors'      : 2,
#         'instrumentSerialNumber' : 'AAA-0001'
#         },
#     'detectorsListHeader' :
#         [
#             {
#             'detectorID'            : 0,
#             'detectorType'          : 'grossGamma',
#             'detectorKind'          : 'naI',
#             'detectorManufacturer'  : 'IEEE',
#             'detectorModel'         : 'Basic Gamma 1.00',
#             'exposureCoeff'         : 7.5,
#             'backgroundSubtraction' : True,
#             'numberOfMCAChannels'   : 1,
#             'calibration' :
#                 {
#                 'calibrationTime' : datetime.fromisoformat('2023-03-20T09:23:27'),
#                 }
#             },
#             {
#             'detectorID'            : 1,
#             'detectorType'          : 'grossNeutron',
#             'detectorKind'          : 'he3',
#             'detectorDescription'   : 'Sample Gross Neutron Counts Detector',
#             'detectorManufacturer'  : 'IEEE',
#             'detectorModel'         : 'Basic Gamma 1.00',
#             'detectorSerialNumber'  : 'N-00000001',
#             'doseConversionCoeff'   : 20,
#             'numberOfMCAChannels'   : 1,
#             },
#         ],
# #     'endHeaderPacket' : 10010
# }

message

{'timeInformationHeader': {'timeStampZero': datetime.datetime(2023, 3, 20, 15, 23, 27),
  'instrumentStatusPacketTransmissionRate': 0.0167},
 'instrumentInformationHeader': {'instrumentName': 'Example SGC Detector',
  'instrumentDescription': 'Sample Spectroscopic Gamma Detector',
  'instrumentManufacturer': 'ANSI',
  'instrumentModel': 'SGC 1.00',
  'instrumentSerialNumber': 'SGC-00000001',
  'numberOfDetectors': 1},
 'detectorsListHeader': [{'detectorID': 0,
   'detectorType': 'spectroscopic',
   'detectorKind': 'cLYC',
   'spectrumCompression': False,
   'backgroundSubtraction': False,
   'numberOfMCAChannels': 1000}]}

In [56]:
encoded = spec.encode('Header', message)

In [57]:
encoded = spec.encode('InstrumentStatus', message)

EncodeError: InstrumentStatus: Sequence member 'instrumentStatusPacketNumber' not found in {'timeInformationHeader': {'timeStampZero': datetime.datetime(2023, 3, 20, 15, 23, 27), 'instrumentStatusPacketTransmissionRate': 0.0167}, 'instrumentInformationHeader': {'instrumentName': 'Example SGC Detector', 'instrumentDescription': 'Sample Spectroscopic Gamma Detector', 'instrumentManufacturer': 'ANSI', 'instrumentModel': 'SGC 1.00', 'instrumentSerialNumber': 'SGC-00000001', 'numberOfDetectors': 1}, 'detectorsListHeader': [{'detectorID': 0, 'detectorType': 'spectroscopic', 'detectorKind': 'cLYC', 'spectrumCompression': False, 'backgroundSubtraction': False, 'numberOfMCAChannels': 1000}]}.

In [61]:
encoded = spec.encode('RadiationData', message)

In [62]:
print(encoded)

bytearray(b'0\x82\x02!\x80\x01\x01\xa1\n\x80\x03\x01\xc5+\x81\x03\x01\xb9\x0f\xa2\x82\x02\x0e0\x82\x02\n\x80\x01\x00\x81\x03\x04*\xa2\xa2\x82\x01\xe6\x02\x02\x02K\x02\x02${\x02\x026_\x02\x02\x19\xbe\x02\x02\x1a\xa5\x02\x02\x1b`\x02\x02\x1c\xc6\x02\x02\x1e\xfa\x02\x02\x1f\xc9\x02\x02\x1fN\x02\x02\x1f\x1c\x02\x02$a\x02\x02*<\x02\x02%"\x02\x02 \xfe\x02\x02\x1d<\x02\x02\x1ae\x02\x02\x18\xaa\x02\x02\x17S\x02\x02\x160\x02\x02\x14\xf3\x02\x02\x14{\x02\x02\x14~\x02\x02\x14\x0f\x02\x02\x13\xeb\x02\x02\x14.\x02\x02\x159\x02\x02\x15\xc9\x02\x02\x16G\x02\x02\x12\xe3\x02\x02\x0b.\x02\x02\x07\x01\x02\x02\x05\x1a\x02\x02\x03\x84\x02\x02\x02\xd8\x02\x02\x02g\x02\x02\x01\xdd\x02\x02\x01\xfe\x02\x02\x03\x13\x02\x02\r&\x02\x029h\x02\x02aN\x02\x02&\xae\x02\x02\x03\xa7\x02\x02\x00\xd6\x02\x02\x00\xbf\x02\x02\x00\xa1\x02\x02\x00\x9d\x02\x02\x00\xab\x02\x02\x00\xbd\x02\x02\x00\xa0\x02\x02\x00\xa9\x02\x02\x00\x87\x02\x02\x00\x90\x02\x01v\x02\x01s\x02\x01o\x02\x01n\x02\x01[\x02\x01Y\x02\x01V\x02\x01@\x02\x01&\

In [63]:
bin(int.from_bytes(encoded, byteorder="big")).strip('0b')

'110000100000100000001000100001100000000000000100000001101000010000101010000000000000110000000111000101001010111000000100000011000000011011100100001111101000101000001000000010000011100011000010000010000000100000101010000000000000010000000010000001000000110000010000101010101000101010001010000010000000011110011000000010000000100000001001001011000000100000001000100100011110110000001000000010001101100101111100000010000000100001100110111110000000100000001000011010101001010000001000000010000110110110000000000010000000100001110011000110000000100000001000011110111110100000001000000010000111111100100100000010000000100001111101001110000000100000001000011111000111000000001000000010001001000110000100000010000000100010101000111100000000100000001000100101001000100000001000000010001000001111111000000010000000100001110100111100000000100000001000011010011001010000001000000010000110001010101000000010000000100001011101010011000000100000001000010110001100000000001000000010000101001111001100000010000000100

In [64]:
print("Length of encoded message:", len(encoded), "bytes")

Length of encoded message: 549 bytes


In [65]:
bin(sum(encoded))[-8:]

'10100111'